In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from shutil import copy2 as copy
from IPython.display import display, HTML
from math import floor, ceil
import json
import re
import os

pd.set_option('display.max_rows', None)

secrets = json.load(open('secrets.json'))
game_version = secrets['game_version']
os.makedirs(game_version, exist_ok=True)

# Sacred Stones Weapons

In [ ]:
wp = pd.read_csv("data/weapons_ss.csv",sep='\s*?,\s*?', engine='python')
wp['Tags'] = wp.Tags.fillna('')

numeric = ['Mt','Wt']
# rng_map = {'1':1, '1~2':3, '2':2, '2~3':5, '1~3':4, '3~10':13, 'All':20, '1~Mag/A':10, '1~Mag/B':10,
#     '1~/Mag/B':10, '1~Mag/C':10, '1~Mag/D':10}
# rng_unmap = {1:'1', 3:'1~2', 2:'2', 5:'2~3', 4:'1~3', 13:'3~10', 20:'All', 10:'1~Str', 10:'1~Str',
#     10:'1~Str', 10:'1~Str', 10:'1~Str'}
# rng_norms = {"Swords": 1, "Lances": 1, "Axes": 1, "Bows": 2, "Reason": 3, "Dark": 3, "Faith": 3}

normal_wp = wp.loc[~wp.Tags.str.contains('Legendary') & (wp.Type != 'Monster')]

wp_tiers = normal_wp[['Name', 'Type']].copy()
for type in set(normal_wp['Type']):
    wp_tiers.loc[wp_tiers.Type == type, 'Mt'] = pd.cut(normal_wp.loc[wp.Type == type, 'Mt'], 5, retbins=True, labels=['0', '1', '2', '3', '4'])[0]
    wp_tiers.loc[wp_tiers.Type == type, 'Wt'] = pd.cut(normal_wp.loc[wp.Type == type, 'Wt'], 5, retbins=True, labels=['-2', '-1', '0', '1', '2'])[0]
wp_tiers['Mt'] = wp_tiers['Mt'].astype('str').str.replace('nan', '0')
wp_tiers['Wt'] = wp_tiers['Wt'].astype('str').str.replace('nan', '0')
wp_tiers = wp_tiers.merge(wp[['Name', 'Rng', 'DmgType', 'Tags']], on='Name').fillna('')

wp_tiers.loc[wp_tiers.Name == 'Iron Sword', 'Mt'] = '1' 
wp_tiers.loc[wp_tiers.Name == 'Shine', 'Wt'] = '-1' 

wp_tiers.sort_values('Mt')
wp_tiers[wp_tiers.Type=='Faith']

In [ ]:
export = wp_tiers.copy()
# export["Atk"] = export.apply(lambda x: x.Mag if x.Mag != '-' else x.Str, axis=1)
# export['HP'] = 20
export_order = ['Name', 'Type', 'Mt', 'Wt', 'Rng', 'DmgType', 'Tags']
export = export[export_order].sort_values('Type')
export.to_csv(f'{game_version}/weapons_ss.csv', index=False)
md_table = export.to_markdown(index=0)
open(f'{game_version}/Weapons.md', 'w').write(md_table)
open(f'{secrets["ObsidianRoot"]}/{game_version}/Weapons.md', 'w').write(md_table)
print(md_table)

# Engage Weapons

In [5]:
def fix(df, check, pattern, target, value):
    df.loc[df[check].str.contains(pattern), target] = value

def adjust(df, check, pattern, target, value):
    df.loc[df[check].str.contains(pattern), target] = df.loc[df[check].str.contains(pattern), target]+value


engage_raw = pd.read_csv("data/engage_weapons_groom.csv",sep='\s*?,\s*?', engine='python')
engage_raw['Tags'] = engage_raw.Tags.fillna('')
numeric = ['Mt', 'Hit', 'Crit', 'Wt']
slicekey = 'Type'

# Raw data adjustments and filters

engage_raw.loc[engage_raw.Name.str.contains("urge"), "Hit"] = 100
# engage_raw.loc[engage_raw.Name.str.contains("Meteor"), "Wt"] = 15
# engage_raw.loc[engage_raw.Wt > 20, "Wt"] = 20

# Rating generation

wp_tiers = engage_raw[['Name', slicekey]].copy()
for type in set(engage_raw[slicekey]):
    
    # Might 
    
    base, rank_climb, type_diff = 7, 2, 2
    wp_tiers.loc[wp_tiers[slicekey] == type, 'Mt'] = engage_raw.loc[wp_tiers[slicekey] == type, 'Lvl'].map({'C':base, 'B':base+rank_climb*1, 'A':base+rank_climb*2, 'S':base+rank_climb*3})
    if type in ['Sword', 'Dagger']: 
        wp_tiers.loc[wp_tiers[slicekey] == type, 'Mt'] = wp_tiers.loc[wp_tiers[slicekey] == type, 'Mt'] - type_diff
    elif type in ['Axe']: 
        wp_tiers.loc[wp_tiers[slicekey] == type, 'Mt'] = wp_tiers.loc[wp_tiers[slicekey] == type, 'Mt'] + type_diff
    elif type in ['Art']: 
        wp_tiers.loc[wp_tiers[slicekey] == type, 'Mt'] = wp_tiers.loc[wp_tiers[slicekey] == type, 'Mt'] - type_diff*3
    
    # Weight
    if type == 'Staff':
        wp_tiers.loc[wp_tiers[slicekey] == type, 'Wt'] = '0'
    else:
        med = min(int(engage_raw.loc[engage_raw[slicekey] == type, 'Wt'].median()), 13)
        high = 14
        
        wp_tiers.loc[wp_tiers[slicekey] == type, 'Wt'] = pd.cut(engage_raw.loc[engage_raw[slicekey] == type, 'Wt'], [0, med, high, 99], retbins=True, labels=['0', '-5', '-10'])[0]
wp_tiers.Mt = wp_tiers.Mt.astype('int')

wp_tiers['Mt'] = wp_tiers['Mt'].astype('str').str.replace('nan', '0')
wp_tiers['Wt'] = wp_tiers['Wt'].astype('str').str.replace('nan', '0')
wp_tiers['Hit'] = engage_raw.Hit.apply(lambda x: -max(80-x, 0)) # Incease _-x to be more penalizing
wp_data = wp_tiers.merge(engage_raw, on=[slicekey, 'Name'], suffixes=[None, 'Value']).fillna('')
wp_data.Mt = wp_data.Mt.astype('int')
wp_data.Wt = wp_data.Wt.astype('int')
wp_data.Hit = wp_data.Hit.astype('int')

# Spot data tuning

lvl_cats = pd.api.types.CategoricalDtype(
    ['S', 'A', 'B', 'C', 'D'], 
    ordered=True
)
type_cats = pd.api.types.CategoricalDtype(
    ['Sword', 'Axe', 'Lance', 'Bow', 'Dagger', 'Art', 'Tome', 'Staff'], 
    ordered=True
)
wp_data.Lvl = wp_data.Lvl.astype(lvl_cats)
wp_data.Type = wp_data.Type.astype(type_cats)
wp_data.loc[wp_data.Name.str.contains("urge"), "Hit"] = 100
wp_data.Price = wp_data.Price/100
adjust(wp_data, 'Name', '(Javelin|Hand Axe)', 'Mt', -2)
adjust(wp_data, 'Name', '(Javelin|Hand Axe)', 'Hit', -5)
adjust(wp_data, 'Name', '(Javelin|Hand Axe)', 'Wt', -5)
wp_data["DmgType"] = wp_data.apply(lambda x: 'Mag' if x.Type in ['Tome', 'Staff'] else 'Phys',axis=1)
wp_data["DmgType"] = wp_data.apply(lambda x: 'Mag' if 'Enchanted' in x.Tags else x.DmgType,axis=1)

# # adjust(wp_data, 'Name', '(Killer Lance|Rider|Hammer|Poleaxe)', 'Mt', 2)
# fix(wp_data, 'Name', 'Radiant', 'Mt', 7)
# # wp_data.loc[wp_data.Type.str.contains('Tome'), 'Mt'] = wp_data.loc[wp_data.Type.str.contains('Tome'), 'Mt'].clip(upper=7)

# # Export 

wp_data.Wt = wp_data.Wt.astype('int')
export = wp_data.copy()
export_order = ['Type', 'Name', 'Lvl', 'Mt', 'Wt', 'Hit', 'Rng', 'DmgType', 'Price', 'Tags']
export = export[export_order].sort_values(['Type', 'Lvl'])

# # ---

export.sort_values(['Type', 'Wt'], ascending=True)
# export.sort_values(['Wt'], ascending=True)
# wp_data[wp_data.Type == 'Sword'].sort_values(['Lvl', 'Mt'], ascending=True)
export[export.Type == 'Art'].set_index(['Lvl', 'Name', 'Mt']).sort_index(level=[0,2])
# export[export.Name.str.contains('Silver|Steel|Iron|Slim|Compact|Initiate|Short')].sort_values(['Type', 'Mt'], ascending=True)
# s_ranks

/var/folders/ks/dksml1ms0rg9dk9rjyqpnfdc0000gn/T/ipykernel_44821/3320905954.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df[check].str.contains(pattern), target] = df.loc[df[check].str.contains(pattern), target]+value


Type  Wt  Hit Rng DmgType  Price  \
Lvl Name              Mt                                    
S   Divine Fist Art   7   Art  -5   -5   1    Phys  100.0   
B   Flashing Fist Art 3   Art  -5    0   1    Phys   60.0   
    Silver-Spirit Art 3   Art   0    0   1    Phys   40.0   
C   Shielding Art     1   Art   0  -10   1    Phys   25.0   
    Steel-Hand Art    1   Art   0  -10   1    Phys   20.0   

                                                                    Tags  
Lvl Name              Mt                                                  
S   Divine Fist Art   7                                            Brave  
B   Flashing Fist Art 3   Brave Bonus(Agi+5) Cursed(Def-2) Cursed(Res-2)  
    Silver-Spirit Art 3                                            Brave  
C   Shielding Art     1                                     Bonus(Def+1)  
    Steel-Hand Art    1                                            Brave

In [12]:
export.to_csv(f'{game_version}/weapons_engage.csv', index=False)
md_table = ''
for g, data in export.groupby(['Type']):
    md_table += f'## {g}\n\n'
    md_table += data.to_markdown(index=0)+'\n\n'
# md_table = export.to_markdown(index=0)
open(f'{game_version}/Weapons.md', 'w').write(md_table)
open(f'{secrets["ObsidianRoot"]}/{game_version}/Weapons.md', 'w').write(md_table)
# print(md_table)

11204